# Python 資料分析

> 以 Pandas 處理表格式資料

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import datetime
import urllib
import requests
import numpy as np

## 關於 Pandas

## 什麼是 Pandas？

> Pandas 是 Python 處理表格式資料的第三方模組（套件），它創造了稱為索引（Index）、序列（Series）與資料框（DataFrame）的資料結構，透過這些資料結構，可以讓 Python 在面對文字檔案、Excel 試算表與關聯式資料庫時能夠使用更直覺的觀念操作，其中資料框啟發自 R 語言。

來源：<https://github.com/pandas-dev/pandas>

## 為什麼資料分析採用 Pandas？

在 Pandas 誕生之前（2008 年）Python 並沒有一個合適的資料結構來處理資料分析常見的「表格式資料（Tabular datasets）」，Python 使用者必須在工作流程中切換到像是 R 語言或者 Matlab。

## 以 `import` 指令載入 Pandas

`pandas` 的官方縮寫為 `pd`。

In [2]:
import pandas as pd

## 如果環境中沒有安裝 Pandas，載入時會遭遇 `ModuleNotFoundError`

```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'pandas'
```

## 如果遭遇 `ModuleNotFoundError` 可以在終端機使用 `pip install` 指令安裝

```bash
pip install pandas
```

## 可以透過兩個屬性檢查版本號與安裝路徑

- `__version__` 屬性檢查版本號。
- `__file__` 屬性檢查安裝路徑。

In [3]:
print(pd.__version__)
print(pd.__file__)

1.2.4
/Users/kuoyaojen/opt/miniconda3/envs/pyds/lib/python3.8/site-packages/pandas/__init__.py


## （沒什麼用的）冷知識：Pandas 跟熊貓的關係是？

![](https://media.giphy.com/media/46Zj6ze2Z2t4k/giphy.gif)

來源：<https://giphy.com>

## 與熊貓完全沒有關係，Pandas 是由三個起初創造的資料結構類別名稱組合而成

- **Pan**el（自從版本 0.20.0 之後停用）
- **Da**taFrame
- **S**eries

## Pandas 起步走

## 想要精通 Pandas，必須掌握 `Index`、`ndarray`、`Series` 與 `DataFrame` 四個類別之間的關係

- `Series` 由 `Index` 與 `ndarray` 組合而成。
- `DataFrame` 由數個共享同一個 `Index` 的 `Series` 組合而成。

## Pandas 的 `Index` 類別

使用 `Index` 函數創造 `Index` 類別的實例。

In [4]:
prime_indices = pd.Index([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
print(type(prime_indices))

<class 'pandas.core.indexes.numeric.Int64Index'>


## `Index` 類別像是 Python 內建 `tuple` 與 `set` 兩種類別的結合

- 具有 `tuple` 無法更動的特性。
- 具有 `set` 集合運算的特性。 

In [5]:
# Index has the characteristics of a tuple
prime_indices = pd.Index([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
try:
    prime_indices[-1] = 31
except:
    print("Immutable!")

Immutable!


In [6]:
# Index has the characteristics of a set
odd_indices = pd.Index(range(1, 30, 2))
print(prime_indices.intersection(odd_indices))         # prime_indices & odd_indices
print(prime_indices.union(odd_indices))                # prime_indices | odd_indices
print(prime_indices.symmetric_difference(odd_indices)) # prime_indices ^ odd_indices
print(prime_indices.difference(odd_indices))
print(odd_indices.difference(prime_indices))

Int64Index([3, 5, 7, 11, 13, 17, 19, 23, 29], dtype='int64')
Int64Index([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29], dtype='int64')
Int64Index([1, 2, 9, 15, 21, 25, 27], dtype='int64')
Int64Index([2], dtype='int64')
Int64Index([1, 9, 15, 21, 25, 27], dtype='int64')


## Pandas 的 `Series` 類別

使用 `Series` 函數創造 `Series` 類別的實例。

In [7]:
prime_series = pd.Series([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
print(type(prime_series))

<class 'pandas.core.series.Series'>


## `Series` 由 `Index` 與 `ndarray` 組合而成

In [8]:
print(type(prime_series.index))
print(type(prime_series.values))

<class 'pandas.core.indexes.range.RangeIndex'>
<class 'numpy.ndarray'>


## Pandas 的 `DataFrame` 類別

使用 `DataFrame` 函數創造 `DataFrame` 類別的實例。

In [9]:
movie_df = pd.DataFrame()
movie_df["title"] = ["The Shawshank Redemption", "The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
movie_df["imdb_rating"] = [9.3, 9.0, 8.9, 8.8, 8.7]
print(type(movie_df))

<class 'pandas.core.frame.DataFrame'>


## `DataFrame` 由數個 `Series` 共享同一個 `Index` 組成

In [10]:
print(type(movie_df.index))
print(type(movie_df["title"]))
print(type(movie_df["imdb_rating"]))

<class 'pandas.core.indexes.range.RangeIndex'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Pandas 提供函數、類別屬性與方法處理表格式資料的

- 載入（Import）
- 清理（Tidy）
- 轉型（Transform）

## `DataFrame` 的基礎屬性和方法

In [11]:
def get_latest_daily_report():
    """
    This function returns the latest global daily report from https://github.com/CSSEGISandData
    /COVID-19 and its file date.
    """
    latest_date = datetime.date.today()
    day_delta = datetime.timedelta(days=1)
    fmt = '%m-%d-%Y'
    while True:
        try:
            latest_date_fmt = latest_date.strftime(fmt)
            csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master\
/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_fmt)
            daily_report = pd.read_csv(csv_url)
            print("載入了 {} 的每日報告。".format(latest_date_fmt))
            break
        except urllib.error.HTTPError:
            latest_date_fmt = latest_date.strftime(fmt)
            print("尚未有 {} 的每日報告。".format(latest_date_fmt))
            latest_date -= day_delta
    return daily_report

In [12]:
daily_report = get_latest_daily_report()

尚未有 08-07-2021 的每日報告。
載入了 08-06-2021 的每日報告。


## `DataFrame` 的基礎屬性

- `shape`
- `dtypes`
- `index`
- `columns`

In [13]:
print(daily_report.shape)
print(daily_report.dtypes)
print(daily_report.index)
print(daily_report.columns)

(3987, 14)
FIPS                   float64
Admin2                  object
Province_State          object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                   int64
Recovered              float64
Active                 float64
Combined_Key            object
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object
RangeIndex(start=0, stop=3987, step=1)
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio'],
      dtype='object')


## `DataFrame` 的基礎方法

- `head(n)`
- `tail(n)`
- `describe`
- `info`
- `set_index`
- `reset_index`

## `head(n)` 方法回傳欄位名稱與前 n 列觀測值

In [14]:
daily_report.head() # n is default to 5

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-08-07 04:21:30,33.93911,67.709953,149810,6879,NaN,NaN,Afghanistan,384.835305,4.591816
1,NaN,NaN,NaN,Albania,2021-08-07 04:21:30,41.15330,20.168300,133591,2458,NaN,NaN,Albania,4642.122455,1.839944
2,NaN,NaN,NaN,Algeria,2021-08-07 04:21:30,28.03390,1.659600,179216,4487,NaN,NaN,Algeria,408.692674,2.503683
3,NaN,NaN,NaN,Andorra,2021-08-07 04:21:30,42.50630,1.521800,14836,128,NaN,NaN,Andorra,19201.449557,0.862766
4,NaN,NaN,NaN,Angola,2021-08-07 04:21:30,-11.20270,17.873900,43487,1042,NaN,NaN,Angola,132.314992,2.396118


## `tail(n)` 方法回傳欄位名稱與後 n 列觀測值

In [15]:
daily_report.tail(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
3984,NaN,NaN,NaN,Yemen,2021-08-07 04:21:30,15.552727,48.516388,7131,1384,NaN,NaN,Yemen,23.908696,19.408218
3985,NaN,NaN,NaN,Zambia,2021-08-07 04:21:30,-13.133897,27.849332,199135,3459,NaN,NaN,Zambia,1083.199938,1.737013
3986,NaN,NaN,NaN,Zimbabwe,2021-08-07 04:21:30,-19.015438,29.154857,115445,3805,NaN,NaN,Zimbabwe,776.731259,3.295942


## `describe` 方法回傳數值欄位的描述性統計

In [16]:
daily_report.describe()

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
count,3266.000000,3899.000000,3899.000000,3.987000e+03,3987.000000,0.0,0.0,3898.000000,3946.000000
mean,32401.272811,35.953350,-71.975511,5.059132e+04,1072.921244,NaN,NaN,9636.251069,3.101322
std,18033.806874,13.073420,53.882250,2.855852e+05,6210.766838,NaN,NaN,4114.457792,66.781560
min,66.000000,-52.368000,-178.116500,0.000000e+00,0.000000,NaN,NaN,0.000000,0.000000
25%,19049.500000,33.265691,-96.619226,1.243000e+03,20.000000,NaN,NaN,7464.094146,1.212252
50%,30068.000000,37.941636,-86.854759,3.430000e+03,61.000000,NaN,NaN,10266.849771,1.737013
75%,47040.500000,42.215069,-77.489691,1.477700e+04,221.000000,NaN,NaN,12258.584041,2.425152
max,99999.000000,71.706900,178.065000,6.336220e+06,140428.000000,NaN,NaN,45099.477400,4193.103448


## `info` 方法回傳資料框的詳細資訊

In [17]:
daily_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3987 entries, 0 to 3986
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FIPS                 3266 non-null   float64
 1   Admin2               3271 non-null   object 
 2   Province_State       3813 non-null   object 
 3   Country_Region       3987 non-null   object 
 4   Last_Update          3987 non-null   object 
 5   Lat                  3899 non-null   float64
 6   Long_                3899 non-null   float64
 7   Confirmed            3987 non-null   int64  
 8   Deaths               3987 non-null   int64  
 9   Recovered            0 non-null      float64
 10  Active               0 non-null      float64
 11  Combined_Key         3987 non-null   object 
 12  Incident_Rate        3898 non-null   float64
 13  Case_Fatality_Ratio  3946 non-null   float64
dtypes: float64(7), int64(2), object(5)
memory usage: 436.2+ KB


## `set_index` 方法可以指定欄位取代目前的 `Index`

In [18]:
daily_report.set_index('Combined_Key')

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
Combined_Key,,,,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,Afghanistan,2021-08-07 04:21:30,33.939110,67.709953,149810,6879,NaN,NaN,384.835305,4.591816
Albania,NaN,NaN,NaN,Albania,2021-08-07 04:21:30,41.153300,20.168300,133591,2458,NaN,NaN,4642.122455,1.839944
Algeria,NaN,NaN,NaN,Algeria,2021-08-07 04:21:30,28.033900,1.659600,179216,4487,NaN,NaN,408.692674,2.503683
Andorra,NaN,NaN,NaN,Andorra,2021-08-07 04:21:30,42.506300,1.521800,14836,128,NaN,NaN,19201.449557,0.862766
Angola,NaN,NaN,NaN,Angola,2021-08-07 04:21:30,-11.202700,17.873900,43487,1042,NaN,NaN,132.314992,2.396118
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,NaN,NaN,NaN,Vietnam,2021-08-07 04:21:30,14.058324,108.277199,193381,3016,NaN,NaN,198.668394,1.559615
West Bank and Gaza,NaN,NaN,NaN,West Bank and Gaza,2021-08-07 04:21:30,31.952200,35.233200,317703,3611,NaN,NaN,6227.741474,1.136596
Yemen,NaN,NaN,NaN,Yemen,2021-08-07 04:21:30,15.552727,48.516388,7131,1384,NaN,NaN,23.908696,19.408218


## `reset_index` 方法可以用預設的 `RangeIndex` 重設目前的 `Index`

In [19]:
daily_report.set_index('Combined_Key').reset_index()

,Combined_Key,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
0,Afghanistan,NaN,NaN,NaN,Afghanistan,2021-08-07 04:21:30,33.939110,67.709953,149810,6879,NaN,NaN,384.835305,4.591816
1,Albania,NaN,NaN,NaN,Albania,2021-08-07 04:21:30,41.153300,20.168300,133591,2458,NaN,NaN,4642.122455,1.839944
2,Algeria,NaN,NaN,NaN,Algeria,2021-08-07 04:21:30,28.033900,1.659600,179216,4487,NaN,NaN,408.692674,2.503683
3,Andorra,NaN,NaN,NaN,Andorra,2021-08-07 04:21:30,42.506300,1.521800,14836,128,NaN,NaN,19201.449557,0.862766
4,Angola,NaN,NaN,NaN,Angola,2021-08-07 04:21:30,-11.202700,17.873900,43487,1042,NaN,NaN,132.314992,2.396118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3982,Vietnam,NaN,NaN,NaN,Vietnam,2021-08-07 04:21:30,14.058324,108.277199,193381,3016,NaN,NaN,198.668394,1.559615
3983,West Bank and Gaza,NaN,NaN,NaN,West Bank and Gaza,2021-08-07 04:21:30,31.952200,35.233200,317703,3611,NaN,NaN,6227.741474,1.136596
3984,Yemen,NaN,NaN,NaN,Yemen,2021-08-07 04:21:30,15.552727,48.516388,7131,1384,NaN,NaN,23.908696,19.408218
3985,Zambia,NaN,NaN,NaN,Zambia,2021-08-07 04:21:30,-13.133897,27.849332,199135,3459,NaN,NaN,1083.199938,1.737013


## 基礎的資料框操作

## Pandas 使用更直觀的概念操作資料

- 如何定義「更直觀」？
    - 像操作試算表一樣（Spreadsheet-like）。
    - 像使用資料庫一樣（SQL-like）。

## 基礎的資料框操作語法

- 選擇。
- 篩選。
- 選擇與篩選。

## 基礎的資料框操作語法（續）

- 索引。
- 排序。
- 衍生計算欄位。
- 聚合。
- 分組聚合。

## 以 `['欄位名稱']` 語法選擇出資料框的欄位成為外型 `(m,)` 的 `Series`

In [20]:
print(daily_report['Country_Region'])
print(type(daily_report['Country_Region']))

0              Afghanistan
1                  Albania
2                  Algeria
3                  Andorra
4                   Angola
               ...        
3982               Vietnam
3983    West Bank and Gaza
3984                 Yemen
3985                Zambia
3986              Zimbabwe
Name: Country_Region, Length: 3987, dtype: object
<class 'pandas.core.series.Series'>


## 以 `[['欄位名稱']]` 語法選擇出資料框的欄位成為外型 `(m, 1)` 的 `DataFrame`

In [21]:
print(type(daily_report[['Country_Region']]))
daily_report[['Country_Region']]

<class 'pandas.core.frame.DataFrame'>


,Country_Region
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
3982,Vietnam
3983,West Bank and Gaza
3984,Yemen
3985,Zambia


## 以 `[['欄位名稱', '欄位名稱', ...]]` 語法選擇出資料框多個欄位成為外型 `(m, n)` 的 `DataFrame`

In [22]:
cols = ['Country_Region', 'Lat', 'Long_']
daily_report[cols]

,Country_Region,Lat,Long_
0,Afghanistan,33.939110,67.709953
1,Albania,41.153300,20.168300
2,Algeria,28.033900,1.659600
3,Andorra,42.506300,1.521800
4,Angola,-11.202700,17.873900
...,...,...,...
3982,Vietnam,14.058324,108.277199
3983,West Bank and Gaza,31.952200,35.233200
3984,Yemen,15.552727,48.516388
3985,Zambia,-13.133897,27.849332


## 透過條件判斷式以 `[布林陣列]` 篩選觀測值

In [23]:
is_taiwan = daily_report['Country_Region'] == 'Taiwan*'
print(is_taiwan)
daily_report[is_taiwan]

0       False
1       False
2       False
3       False
4       False
        ...  
3982    False
3983    False
3984    False
3985    False
3986    False
Name: Country_Region, Length: 3987, dtype: bool


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
647,NaN,NaN,NaN,Taiwan*,2021-08-07 04:21:30,23.7,121.0,15765,794,NaN,NaN,Taiwan*,66.192841,5.036473


## 同時選擇欄位以及篩選觀測值

In [24]:
cols_to_select = ['Country_Region', 'Confirmed']
rows_to_filter = daily_report['Country_Region'] == 'Taiwan*'
daily_report[rows_to_filter][cols_to_select]

,Country_Region,Confirmed
647,Taiwan*,15765


## 運用 `DataFrame` 的兩種索引語法取得資料

- `loc[index_label, column_label]`
- `iloc[index_position, column_position]`

## `loc[]` 以資料的列、欄標籤為準 

In [25]:
tokyo_loc = (daily_report[daily_report['Province_State'] == 'Tokyo']).index[0]
jp = daily_report[daily_report['Country_Region'] == 'Japan']
print(tokyo_loc)
jp.loc[tokyo_loc, ['Province_State', 'Country_Region', 'Confirmed']]

354


Province_State     Tokyo
Country_Region     Japan
Confirmed         240721
Name: 354, dtype: object

## `iloc[]` 以資料框的列、欄位置為準 

In [26]:
tokyo_iloc = [i for i, v in enumerate(jp.index) if v == tokyo_loc][0]
print(tokyo_iloc)
jp.iloc[tokyo_iloc, [2, 3, 7]]

41


Province_State     Tokyo
Country_Region     Japan
Confirmed         240721
Name: 354, dtype: object

## 使用（呼叫）資料框的兩種方法排序

- `sort_values`
- `sort_index`

## `sort_values` 方法可依指定欄位排序資料框

In [27]:
daily_report.sort_values('Confirmed')

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
141,NaN,NaN,Unknown,China,2021-08-07 04:21:30,NaN,NaN,0,0,NaN,NaN,"Unknown, China",NaN,NaN
619,NaN,NaN,Unknown,Spain,2021-04-25 23:20:49,NaN,NaN,0,0,NaN,NaN,"Unknown, Spain",NaN,NaN
3550,90048.0,Unassigned,Texas,US,2021-08-07 04:21:30,NaN,NaN,0,0,NaN,NaN,"Unassigned, Texas, US",NaN,NaN
2829,90039.0,Unassigned,Ohio,US,2021-08-07 04:21:30,NaN,NaN,0,4,NaN,NaN,"Unassigned, Ohio, US",NaN,NaN
3103,NaN,NaN,Recovered,US,2021-08-07 04:21:30,NaN,NaN,0,0,NaN,NaN,"Recovered, US",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,NaN,NaN,NaN,Argentina,2021-08-07 04:21:30,-38.416100,-63.616700,5002951,107213,NaN,NaN,Argentina,11069.509879,2.142995
3967,NaN,NaN,England,United Kingdom,2021-08-07 04:21:30,52.355500,-1.174300,5253181,114359,NaN,NaN,"England, United Kingdom",9384.501190,2.176948
655,NaN,NaN,NaN,Turkey,2021-08-07 04:21:30,38.963700,35.243300,5870741,51976,NaN,NaN,Turkey,6960.879707,0.885340
216,NaN,NaN,NaN,France,2021-08-07 04:21:30,46.227600,2.213700,6161967,111050,NaN,NaN,France,9440.225922,1.802184


## `sort_index` 可依資料框的 `Index` 排序 

In [28]:
daily_report.sort_index(ascending=False)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
3986,NaN,NaN,NaN,Zimbabwe,2021-08-07 04:21:30,-19.015438,29.154857,115445,3805,NaN,NaN,Zimbabwe,776.731259,3.295942
3985,NaN,NaN,NaN,Zambia,2021-08-07 04:21:30,-13.133897,27.849332,199135,3459,NaN,NaN,Zambia,1083.199938,1.737013
3984,NaN,NaN,NaN,Yemen,2021-08-07 04:21:30,15.552727,48.516388,7131,1384,NaN,NaN,Yemen,23.908696,19.408218
3983,NaN,NaN,NaN,West Bank and Gaza,2021-08-07 04:21:30,31.952200,35.233200,317703,3611,NaN,NaN,West Bank and Gaza,6227.741474,1.136596
3982,NaN,NaN,NaN,Vietnam,2021-08-07 04:21:30,14.058324,108.277199,193381,3016,NaN,NaN,Vietnam,198.668394,1.559615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,NaN,NaN,NaN,Angola,2021-08-07 04:21:30,-11.202700,17.873900,43487,1042,NaN,NaN,Angola,132.314992,2.396118
3,NaN,NaN,NaN,Andorra,2021-08-07 04:21:30,42.506300,1.521800,14836,128,NaN,NaN,Andorra,19201.449557,0.862766
2,NaN,NaN,NaN,Algeria,2021-08-07 04:21:30,28.033900,1.659600,179216,4487,NaN,NaN,Algeria,408.692674,2.503683
1,NaN,NaN,NaN,Albania,2021-08-07 04:21:30,41.153300,20.168300,133591,2458,NaN,NaN,Albania,4642.122455,1.839944


## 從資料框變數衍生計算

- 簡單運算。
- 使用 Pandas 的 `cut` 函數。
- 使用 `Series` 的 `map` 方法。

## 簡單運算

In [29]:
active = daily_report['Confirmed'] - daily_report['Deaths'] - daily_report['Recovered']
print(active)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3982   NaN
3983   NaN
3984   NaN
3985   NaN
3986   NaN
Length: 3987, dtype: float64


## 使用 Pandas 的 `cut` 函數

In [30]:
cut_bins = [0, 1000, 10000, 100000, np.Inf]
cut_labels = ['Less than 1000', 'Between 1000 and 10000', 'Between 10000 and 100000', 'Above 100000']
confirmed_categorical = pd.cut(daily_report['Confirmed'], bins=cut_bins, labels=cut_labels, right=False)
print(confirmed_categorical)

0                   Above 100000
1                   Above 100000
2                   Above 100000
3       Between 10000 and 100000
4       Between 10000 and 100000
                  ...           
3982                Above 100000
3983                Above 100000
3984      Between 1000 and 10000
3985                Above 100000
3986                Above 100000
Name: Confirmed, Length: 3987, dtype: category
Categories (4, object): ['Less than 1000' < 'Between 1000 and 10000' < 'Between 10000 and 100000' < 'Above 100000']


## 使用 `Series` 的 `map` 方法

- 傳入一個 `dict`。
- 傳入一個函數（或 Lambda 函數）。

In [31]:
# Passing a dict
country_name = {
    'Taiwan*': 'Taiwan'
}
daily_report_tw = daily_report[is_taiwan]
daily_report_tw['Country_Region'].map(country_name)

647    Taiwan
Name: Country_Region, dtype: object

In [32]:
# Passing a function
def is_us(x):
    if x == 'US':
        return 'US'
    else:
        return 'Not US'
daily_report['Country_Region'].map(is_us)

0       Not US
1       Not US
2       Not US
3       Not US
4       Not US
         ...  
3982    Not US
3983    Not US
3984    Not US
3985    Not US
3986    Not US
Name: Country_Region, Length: 3987, dtype: object

In [33]:
# Passing a lambda expression)
daily_report['Country_Region'].map(lambda x: 'US' if x == 'US' else 'Not US')

0       Not US
1       Not US
2       Not US
3       Not US
4       Not US
         ...  
3982    Not US
3983    Not US
3984    Not US
3985    Not US
3986    Not US
Name: Country_Region, Length: 3987, dtype: object

## 如何新增資料框的變數

## 使用（呼叫）`Series` 的聚合方法摘要

In [34]:
daily_report['Confirmed'].sum()

201707586

## 使用（呼叫）`Series` 的 `groupby` 方法分組聚合

In [35]:
daily_report.groupby('Country_Region')['Confirmed'].sum()

Country_Region
Afghanistan           149810
Albania               133591
Algeria               179216
Andorra                14836
Angola                 43487
                       ...  
Vietnam               193381
West Bank and Gaza    317703
Yemen                   7131
Zambia                199135
Zimbabwe              115445
Name: Confirmed, Length: 195, dtype: int64

## 進階的資料框操作

## 進階的資料框操作包含

- 處理遺漏值（未定義值）。
- 處理文字變數。
- 轉置。
- 合併資料框。

## 處理遺漏值（未定義值）

- 使用（呼叫）`Series` 的 `isnull` 或者 `notnull` 方法來檢查遺漏值是否存在。
- 使用（呼叫）`DataFrame` 的 `dropna` 方法刪除有遺漏值的觀測值或欄位。
- 使用（呼叫）`Series` 的 `fillna` 方法填補遺漏值為指定資料。

In [36]:
print(daily_report['Province_State'].size)
print(daily_report['Province_State'].isnull().sum())
print(daily_report['Province_State'].notnull().sum())

3987
174
3813


In [37]:
print(daily_report.dropna().shape)
print(daily_report['FIPS'].fillna(0))

(0, 14)
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3982    0.0
3983    0.0
3984    0.0
3985    0.0
3986    0.0
Name: FIPS, Length: 3987, dtype: float64


## `Series` 摘要文字變數的方法

- `unique`
- `nunique`
- `value_counts`

In [38]:
print(daily_report['Country_Region'].nunique())
print(daily_report['Country_Region'].unique())

195
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Comoros' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan'

In [39]:
daily_report['Country_Region'].value_counts().sort_values(ascending=False)

US            3277
Russia          83
Japan           49
India           37
Colombia        34
              ... 
Thailand         1
Tanzania         1
Bangladesh       1
Poland           1
Madagascar       1
Name: Country_Region, Length: 195, dtype: int64

## 使用（呼叫）`Series` 的 `str.split` 方法切割文字

In [40]:
request_url = "https://www.imdb.com/chart/top"
html_tables = pd.read_html(request_url)
html_table = html_tables[0]
html_table.head()

,Unnamed: 0,Rank & Title,IMDb Rating,Your Rating,Unnamed: 4
0,NaN,1. 刺激1995 (1994),9.2,12345678910 NOT YET RELEASED Seen,NaN
1,NaN,2. 教父 (1972),9.1,12345678910 NOT YET RELEASED Seen,NaN
2,NaN,3. 教父第二集 (1974),9.0,12345678910 NOT YET RELEASED Seen,NaN
3,NaN,4. 黑暗騎士 (2008),9.0,12345678910 NOT YET RELEASED Seen,NaN
4,NaN,5. 十二怒漢 (1957),8.9,12345678910 NOT YET RELEASED Seen,NaN


In [41]:
html_table['Rank & Title'].str.split() # split as a Series

0        [1., 刺激1995, (1994)]
1            [2., 教父, (1972)]
2         [3., 教父第二集, (1974)]
3          [4., 黑暗騎士, (2008)]
4          [5., 十二怒漢, (1957)]
                ...          
245        [246., 日出, (1927)]
246     [247., 巴黎，德州, (1984)]
247    [248., 卡比莉亞之夜, (1957)]
248    [249., 7號房的禮物, (2019)]
249     [250., 誘惑的陷阱, (2000)]
Name: Rank & Title, Length: 250, dtype: object

In [42]:
html_table['Rank & Title'].str.split(expand=True) # split as a DataFrame

,0,1,2
0,1.,刺激1995,(1994)
1,2.,教父,(1972)
2,3.,教父第二集,(1974)
3,4.,黑暗騎士,(2008)
4,5.,十二怒漢,(1957)
...,...,...,...
245,246.,日出,(1927)
246,247.,巴黎，德州,(1984)
247,248.,卡比莉亞之夜,(1957)
248,249.,7號房的禮物,(2019)


## 使用（呼叫）`Series` 的 `str.replace` 方法取代文字中的指定特徵

In [43]:
split_dataframe = html_table['Rank & Title'].str.split(expand=True)
ranks = split_dataframe[0].str.replace(".", '', regex=False)
ranks

0        1
1        2
2        3
3        4
4        5
      ... 
245    246
246    247
247    248
248    249
249    250
Name: 0, Length: 250, dtype: object

In [44]:
split_dataframe = html_table['Rank & Title'].str.split(expand=True)
years = split_dataframe[2].str.replace("\(|\)", '', regex=True)
years

0      1994
1      1972
2      1974
3      2008
4      1957
       ... 
245    1927
246    1984
247    1957
248    2019
249    2000
Name: 2, Length: 250, dtype: object

## 使用（呼叫）`Series` 的 `str.contains` 方法判斷文字是否包含指定特徵

In [45]:
print(daily_report['Country_Region'].str.contains('land').sum())
daily_report[daily_report['Country_Region'].str.contains('land')].head()

27


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
204,NaN,NaN,NaN,Finland,2021-08-07 04:21:30,61.92411,25.748151,110720,986,NaN,NaN,Finland,1998.296972,0.890535
248,NaN,NaN,NaN,Iceland,2021-08-07 04:21:30,64.96310,-19.020800,8613,30,NaN,NaN,Iceland,2523.956044,0.348311
289,NaN,NaN,NaN,Ireland,2021-08-07 04:21:30,53.14240,-7.692100,308800,5044,NaN,NaN,Ireland,6253.802304,1.633420
386,NaN,NaN,NaN,Marshall Islands,2021-08-07 04:21:30,7.13150,171.184500,4,0,NaN,NaN,Marshall Islands,6.847791,0.000000
431,NaN,NaN,Aruba,Netherlands,2021-08-07 04:21:30,12.52110,-69.968300,12262,111,NaN,NaN,"Aruba, Netherlands",11484.929659,0.905236


## 轉置資料框的函數與方法

- Pandas 的 `melt` 函數可以將資料框由寬格式（Wide format）轉換為長格式（Long format）
- `DataFrame` 的 `pivot` 方法可以將資料框由長格式（Long format）轉換為寬格式（Wide format）。

## 什麼是寬格式、長格式？

- 寬格式使用一個欄位，欄位名稱記錄變數類別、觀測值記錄其數值。 
- 長格式使用兩個欄位，一個記錄變數類別、一個記錄數值。

來源：<https://en.wikipedia.org/wiki/Wide_and_narrow_data>

## 為何需要轉置資料框？

資料框的欄位名稱含有使用者需要的資料值或者儲存格式與應用情境不符。

In [46]:
ts_confirmed_global_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
ts_confirmed_global = pd.read_csv(ts_confirmed_global_url)
ts_confirmed_global

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,145552,145996,146523,146523,146523,147985,148572,148933,149361,149810
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,132952,132999,133036,133081,133121,133146,133211,133310,133442,133591
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,167131,168668,170189,171392,172564,173922,175229,176724,178013,179216
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,14586,14655,14678,14678,14678,14747,14766,14797,14809,14836
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,42288,42486,42646,42777,42815,42970,43070,43158,43269,43487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,123640,133405,141122,150060,157507,157507,174461,181756,189066,193381
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,316328,316487,316603,316738,316861,317083,317264,317404,317534,317703
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,7027,7042,7058,7061,7070,7081,7086,7096,7104,7131
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,193432,194140,195096,195816,196293,196490,197123,197791,198455,199135


## 使用（呼叫）Pandas 的 `melt` 函數寬轉長

- `id_vars`: `['Province/State', 'Country/Region', 'Lat', 'Long']`
- `value_vars`: 從 `1/22/20` 開始到最後一個欄位。
- `var_name`: 自訂為 `Date`。
- `value_name`: 自訂為 `Confirmed`。

In [47]:
idVars = ['Province/State', 'Country/Region', 'Lat', 'Long']
ts_confirmed_global_long = pd.melt(ts_confirmed_global,
                                  id_vars=idVars,
                                  var_name='Date',
                                  value_name='Confirmed')
ts_confirmed_global_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
157072,NaN,Vietnam,14.058324,108.277199,8/6/21,193381
157073,NaN,West Bank and Gaza,31.952200,35.233200,8/6/21,317703
157074,NaN,Yemen,15.552727,48.516388,8/6/21,7131
157075,NaN,Zambia,-13.133897,27.849332,8/6/21,199135


In [48]:
ts_groupby_confirmed = ts_confirmed_global_long.groupby(['Country/Region', 'Date'])['Confirmed'].sum()
ts_groupby_confirmed = pd.DataFrame(ts_groupby_confirmed).reset_index()
ts_groupby_confirmed

,Country/Region,Date,Confirmed
0,Afghanistan,1/1/21,51526
1,Afghanistan,1/10/21,53489
2,Afghanistan,1/11/21,53538
3,Afghanistan,1/12/21,53584
4,Afghanistan,1/13/21,53584
...,...,...,...
109780,Zimbabwe,9/5/20,6837
109781,Zimbabwe,9/6/20,6837
109782,Zimbabwe,9/7/20,7298
109783,Zimbabwe,9/8/20,7388


## 使用（呼叫）`DataFrame` 的 `pivot` 方法長轉寬

In [49]:
ts_groupby_confirmed.pivot(index='Country/Region', columns='Date', values='Confirmed')

Date,1/1/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,...,9/28/20,9/29/20,9/3/20,9/30/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,51526,53489,53538,53584,53584,53775,53831,53938,53984,54062,...,39239,39254,38288,39268,38304,38324,38398,38494,38520,38544
Albania,58316,63595,63971,64627,65334,65994,66635,67216,67690,67982,...,13391,13518,9844,13649,9967,10102,10255,10406,10553,10704
Algeria,99897,102144,102369,102641,102860,103127,103381,103611,103833,104092,...,51213,51368,45469,51530,45773,46071,46364,46653,46938,47216
Andorra,8117,8586,8586,8682,8818,8868,8946,9038,9083,9083,...,1966,1966,1199,2050,1215,1215,1215,1261,1261,1301
Angola,17568,18193,18254,18343,18425,18613,18679,18765,18875,18926,...,4797,4905,2805,4972,2876,2935,2965,2981,3033,3092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,1474,1514,1515,1520,1521,1531,1536,1537,1537,1539,...,1077,1094,1046,1094,1049,1049,1049,1049,1054,1059
West Bank and Gaza,139223,147400,148171,148968,149769,150505,151142,151569,152031,152555,...,39121,39541,24471,39899,25142,25575,26127,26779,27363,27919
Yemen,2101,2104,2105,2107,2109,2110,2111,2112,2112,2113,...,2031,2031,1979,2034,1983,1983,1987,1989,1994,1999


## 合併資料框的函數與方法

- 使用（呼叫）Pandas 的 `merge` 函數利用欄位名稱合併。
- 使用（呼叫）`DataFrame` 的 `join` 方法利用資料框的列索引合併。

In [50]:
def get_top_rated_movies():
    request_url = "https://www.imdb.com/chart/top"
    html_tables = pd.read_html(request_url)
    html_table = html_tables[0]
    split_dataframe = html_table['Rank & Title'].str.split(expand=True)
    ranks = split_dataframe[0].str.replace(".", '', regex=False).values
    years = split_dataframe[2].str.replace("\(|\)", '', regex=True).values
    titles = split_dataframe[1].values
    ratings = html_table['IMDb Rating'].values
    out = pd.DataFrame()
    out['rank'] = ranks
    out['title'] = titles
    out['year'] = years
    out['rating'] = ratings
    return out

top_rated_movies = get_top_rated_movies()

In [51]:
left_df = top_rated_movies.loc[:9, ['title', 'year']]
right_df = top_rated_movies[top_rated_movies['title'].str.contains('魔戒')][['title', 'rating']].reset_index(drop=True)
left_df

,title,year
0,刺激1995,1994
1,教父,1972
2,教父第二集,1974
3,黑暗騎士,2008
4,十二怒漢,1957
5,辛德勒的名單,1993
6,魔戒三部曲：王者再臨,2003
7,黑色追緝令,1994
8,黃昏三鏢客,1966
9,魔戒首部曲：魔戒現身,2001


In [52]:
right_df

,title,rating
0,魔戒三部曲：王者再臨,8.9
1,魔戒首部曲：魔戒現身,8.8
2,魔戒二部曲：雙城奇謀,8.7


## 使用（呼叫）Pandas 的 `merge` 函數利用欄位名稱合併

In [53]:
# default: inner join
pd.merge(left_df, right_df)

,title,year,rating
0,魔戒三部曲：王者再臨,2003,8.9
1,魔戒首部曲：魔戒現身,2001,8.8


In [54]:
# left join
pd.merge(left_df, right_df, how='left')

,title,year,rating
0,刺激1995,1994,NaN
1,教父,1972,NaN
2,教父第二集,1974,NaN
3,黑暗騎士,2008,NaN
4,十二怒漢,1957,NaN
5,辛德勒的名單,1993,NaN
6,魔戒三部曲：王者再臨,2003,8.9
7,黑色追緝令,1994,NaN
8,黃昏三鏢客,1966,NaN
9,魔戒首部曲：魔戒現身,2001,8.8


In [55]:
# right join
pd.merge(left_df, right_df, how='right')

,title,year,rating
0,魔戒三部曲：王者再臨,2003,8.9
1,魔戒首部曲：魔戒現身,2001,8.8
2,魔戒二部曲：雙城奇謀,NaN,8.7


In [56]:
# outer join
pd.merge(left_df, right_df, how='outer')

,title,year,rating
0,刺激1995,1994,NaN
1,教父,1972,NaN
2,教父第二集,1974,NaN
3,黑暗騎士,2008,NaN
4,十二怒漢,1957,NaN
5,辛德勒的名單,1993,NaN
6,魔戒三部曲：王者再臨,2003,8.9
7,黑色追緝令,1994,NaN
8,黃昏三鏢客,1966,NaN
9,魔戒首部曲：魔戒現身,2001,8.8


## 使用（呼叫）DataFrame 的 `join` 方法利用資料框的列索引合併

In [57]:
# join dataframes on index
left_df.join(right_df, lsuffix='_x', rsuffix='_y')

,title_x,year,title_y,rating
0,刺激1995,1994,魔戒三部曲：王者再臨,8.9
1,教父,1972,魔戒首部曲：魔戒現身,8.8
2,教父第二集,1974,魔戒二部曲：雙城奇謀,8.7
3,黑暗騎士,2008,NaN,NaN
4,十二怒漢,1957,NaN,NaN
5,辛德勒的名單,1993,NaN,NaN
6,魔戒三部曲：王者再臨,2003,NaN,NaN
7,黑色追緝令,1994,NaN,NaN
8,黃昏三鏢客,1966,NaN,NaN
9,魔戒首部曲：魔戒現身,2001,NaN,NaN


In [58]:
left_df = left_df.set_index('title')
right_df = right_df.set_index('title')
# default: left join
left_df.join(right_df)

,year,rating
title,,
刺激1995,1994,NaN
教父,1972,NaN
教父第二集,1974,NaN
黑暗騎士,2008,NaN
十二怒漢,1957,NaN
辛德勒的名單,1993,NaN
魔戒三部曲：王者再臨,2003,8.9
黑色追緝令,1994,NaN
黃昏三鏢客,1966,NaN


In [59]:
# inner join
left_df.join(right_df, how='inner')

,year,rating
title,,
魔戒三部曲：王者再臨,2003,8.9
魔戒首部曲：魔戒現身,2001,8.8


In [60]:
# right join
left_df.join(right_df, how='right')

,year,rating
title,,
魔戒三部曲：王者再臨,2003,8.9
魔戒首部曲：魔戒現身,2001,8.8
魔戒二部曲：雙城奇謀,NaN,8.7


In [61]:
# outer join
left_df.join(right_df, how='outer')

,year,rating
title,,
刺激1995,1994,NaN
十二怒漢,1957,NaN
教父,1972,NaN
教父第二集,1974,NaN
辛德勒的名單,1993,NaN
魔戒三部曲：王者再臨,2003,8.9
魔戒二部曲：雙城奇謀,NaN,8.7
魔戒首部曲：魔戒現身,2001,8.8
黃昏三鏢客,1966,NaN
